Árboles de decisión
================

- Uno de los métodos más utilizados.

- Fáciles y rápidos de construir.

- Modelo no-paramétrico: vamos a definir un montón de hiper-parámetros pero el modelo en sí no va a tener pesos ni nada de eso.

- **White-box model**: permiten inferir conocimiento de qué es lo que está pasando.

- El proceso de construcción del árbol realiza un filtrado de variables (estilo embebbed).

- El árbol puede ser expresado como un conjunto de reglas "if-then".

- Podemos utilizarlos para clasificación y regresión.

(Inicialmente nos vamos a concentrar en problemas de clasificación)

# Representación del árbol

- Se compone de nodos y aristas entre los mismos
- Un nodo puede ser de tipo hoja o de decisión
- Si se trata de un nodo de decisión, entonces especificará una condición/prueba contra algún atributo (variable de entrada)
- Un nodo de decisión tendrá tantas aristas salientes como posibles valores tenga el atributo utilizado en la condición que codifica
- Cada nodo hoja va a tener una etiqueta o clase determinada

![](files/images/iris.PNG)

# Ejemplo: ayudando a fisa a encontrar una estrella

In [1]:
import pandas as pd
df = pd.read_csv('data/astronomia.csv', sep=';')
df

,dia,nubosidad,temperatura,humedad,viento,estrellas_visibles
0,1,soleado,caluroso,alta,leve,no
1,2,soleado,caluroso,alta,fuerte,no
2,3,nublado,caluroso,alta,leve,si
3,4,lluvia,templado,alta,leve,si
4,5,lluvia,frio,normal,leve,si
5,6,lluvia,frio,normal,fuerte,no
6,7,nublado,frio,normal,fuerte,si
7,8,soleado,templado,alta,leve,no
8,9,soleado,frio,normal,leve,si
9,10,lluvia,templado,normal,leve,si


![](files/images/decision_tree_1.PNG)

### ¿Cómo clasificamos la siguiente instancia?:

nubosidad = soleado, temperatura = alta, humedad = alta, viento = fuerte

# Pero, ¿árbol o conjunto de reglas?

** En realidad un árbol es una disyunción de conjunciones! **

- Un camino del árbol es una ** conjunción** de pruebas sobre distintos atributos

- El árbol es una ** disyunción ** de esas conjunciones

## Ergo, un árbol es un conjunto de reglas:

(nubosidad=soleado AND humedad=normal) OR (nubosidad=nublado) OR (nubosidad=lluvia AND viento=leve)

# Conjunto completo de reglas

- R1: Si (nubosidad=soleado) Y (humedad=alta) entonces estrellas_visibles=No
- R2: Si (nubosidad=soleado) Y (humedad=normal) entonces estrellas_visibles=Yes
- R3: Si (nubosidad=nublado) entonces estrellas_visibles=Yes
- R4: Si (nubosidad=lluvia) Y (viento=fuerte) entonces estrellas_visibles=No
- R5: Si (nubosidad=lluvia) Y (viento=leve) entonces estrellas_visibles=Yes

# Aprendiendo un árbol ...

Existen muchos algoritmos distintos (CLS, ID3, C4.5, ID4, ID5, C4.8, C5.0)

# Algoritmo básico: ID3 (Iterative Dicotomiser)

- Búsqueda voraz por el espacio de todos los árboles de clasificación posibles
- Enfoque top-down (el árbol se constuye desde la raíz hacia las hojas)
- Para cada nodo se pregunta: qué atributo debería ser utilizado en este momento?
- Crea una rama por cada valor posible de ese atributo
- Se repite el proceso sacando del conjunto de atributos los que ya se utilizaron anteriormente en esa rama
- Termina cuando todas las instancias pertenecen a la misma clase o todos los atributos fueron utilizados
- Se etiqueta los nodos hoja usando la mayoría de las instancias que caen dentro de ese camino

In [2]:
df

,dia,nubosidad,temperatura,humedad,viento,estrellas_visibles
0,1,soleado,caluroso,alta,leve,no
1,2,soleado,caluroso,alta,fuerte,no
2,3,nublado,caluroso,alta,leve,si
3,4,lluvia,templado,alta,leve,si
4,5,lluvia,frio,normal,leve,si
5,6,lluvia,frio,normal,fuerte,no
6,7,nublado,frio,normal,fuerte,si
7,8,soleado,templado,alta,leve,no
8,9,soleado,frio,normal,leve,si
9,10,lluvia,templado,normal,leve,si


In [ ]:
# generamos el nodo raíz
initial = Node(node_type=None,  # es hoja o es de decisión? no sabemos todavia
               remaining_attributes=dataset.columns_names,
               data=dataset)

# y lo agregamos a la lista de pendientes
to_process.append(initial)

while to_process:
    # sacamos algún nodo pendiente de procesar
    current_node = to_process.pop()
    
    # verificamos si tenemos que convertirlo en un nodo hoja, o en un nodo de decisión
    if current_node.data.has_only_one_class() or len(current_node.remaining_attributes) == 0:
        # es un nodo hoja (o todos ejemplos con la misma clase, o no hay más atributos para decidir)
        current_node.node_type = "Leaf, return: " + current_node.get_most_common_class()
    else:
        # es un nodo de decisión!
        
        # determinamos el mejor atributo dentro de los que no fueron evaluados en ese camino.
        best_attribute = get_best_attribute_to_test(current_node.remaining_attributes,
                                                    current_node.data)
        # ahora sabemos que este nodo elige por ese campo
        current_node.node_type = "Condition, based on: " + best_attribute
        
        # y la decisión nos lleva a nodos hijos, uno por cada valor posible del atributo elegido
        for possible_value in current_node.data[best_attribute].unique_values():
            child = Node(node_type=None, # no sabemos todavia si el hijo es hoja u otra decisión
                         remaining_attributes=current_node.remaining_attributes - best_attribute,
                         data=current_node.data.filter(best_attribute, possible_value))
            # lo agregamos como hijo del nodo actual
            current_node.children.append(child)
            # y también lo agregamos a la lista de pendientes, para que lo mire en alguna 
            # iteración próxima
            to_process.append(child)

![](files/images/decision_tree_1.PNG)

## ¿Cómo elegir cuál es el mejor atributo en cada caso?

Queremos elegir el atributo que haga que los datos se separen lo mejor posible !

Para eso usamos el concepto de Información Mutua:

$$ I(C, X_i) = H(C) - H(C | X_i) $$

donde

$$ H(C) = - \sum_{c} p(c) \log_2 p(c) $$
$$ H(C|X)= - \sum_{c} \sum_{x} p(x,c) \log_2 p(c|x) $$

![](files/images/meme_dog.gif)

# Ejemplo: lluvia en Londres?

Estamos dentro de un cine en Londres, y nos preguntamos sobre la probabilidad de que esté lloviendo afuera (C=llueve). Asumamos que la probabilidad de lluvia en dicha ciudad es de 0.5 para un día determinado, entonces H(C)=1. Podemos pensar que dicha variable se encuentra en un grado de incertidumbre máximo dado que cualquier valor tiene la misma probabilidad.

Supongamos que terminó la película y pudimos salir del cine, y somos capaces de ver por nosotros mismos si está lloviendo o no (X=comprobamos visualmente si llueve). En este caso, teniendo la evidencia visual, la variable C está totalmente relacionada con el valor de X; en este caso, podemos decir entonces que H(C|X)=0.

Dicho de otra manera, sabiendo el valor de X (está lloviendo o no) la incertidumbre de C es nula.

## Cómo se relaciona esto con nuestro problema de encontrar qué variable utilizar?

Supongamos ahora que C es nuestra variable a predecir, y que X es alguna de nuestras variables de entrada. 

Si calculo la entropía de C por sí misma, y luego condicionada a una variable de entrada, podría tener algún indicador de qué tanta incertidumbre estoy evitando.

En base a esto, si H(C) = 0.75 y
- H(C|X1) = 0.5
- H(C|X2) = 0.4

Podemos pensar que X2 reduce la incertidumbre de C en mayor medida. Esa disminución es lo que mide la Información Mutua.

In [4]:
df

,dia,nubosidad,temperatura,humedad,viento,estrellas_visibles
0,1,soleado,caluroso,alta,leve,no
1,2,soleado,caluroso,alta,fuerte,no
2,3,nublado,caluroso,alta,leve,si
3,4,lluvia,templado,alta,leve,si
4,5,lluvia,frio,normal,leve,si
5,6,lluvia,frio,normal,fuerte,no
6,7,nublado,frio,normal,fuerte,si
7,8,soleado,templado,alta,leve,no
8,9,soleado,frio,normal,leve,si
9,10,lluvia,templado,normal,leve,si


# Con qué atributo (variable) empezamos ?

## Viento ?

$$ H(C) = -p(si) \log_2 p(si) - p(no) \log_2 p(no) =  -\frac{9}{14} \log_2 \frac{9}{14} - \frac{5}{14} \log_2 \frac{5}{14} = 0.94 $$


$$ H(C|Viento) = -p(fuerte, si) \log_2 p(si|fuerte) - p(fuerte, no) \log_2 p(no|fuerte) - p(leve, si) \log_2 p(si|leve) - p(debil, no) log_2 p(no|debil) = $$
$$ - \frac{3}{14} \log_2 \frac{3}{6} - \frac{3}{14} \log_2 \frac{3}{6} - \frac{6}{14} \log_2 \frac{6}{8} - \frac{2}{14} \log_2 \frac{2}{8} = 0.892 $$


I(C, Viento) = 0.94 - 0.892 = 0.048

## De la misma forma,

I(C, Humedad) = 0.151

**I(C, Nubosidad) = 0.246**

I(C, Temperatura) = 0.029

![](files/images/ID3_1.PNG)

![](files/images/ID3_2.PNG)

Si el valor de **nubosidad** es "nublado", todas las instancias son positivas!. 

Lo transformamos entonces en un nodo hoja, al que le asignamos esa misma etiqueta para clasificar.

## Algunas observaciones ...

- El algoritmo no asegura encontrar el óptimo global
- La complejidad se incrementa de forma lineal con el número de instancias de entrenamiento y de forma exponencial en relación al número de atributos
- Variables con mayor cantidad de valores son favorecidas en la selección (ligado al cálculo de entropía)

# Cuándo?

- Me interesa dar explicaciones de las salidas que da el modelo
- Prototipo rápido: no necesito demasiado pre-procesado
- Dominios complejos donde no existe una clara separación lineal en los datos
- Si tengo que predecir rápido (≈ log(n_train))
- Tengo variables categóricas y numéricas
- Cuando tengo más de dos clases, o múltiples etiquetas!

# Mejoras posibles ...

- ** Establecer una profundidad máxima para el árbol **
- Manejo de variables continuas
- Permitir valores nulos en los datos de entrada
- Definir pesos a las clases o etiquetas (problemas cost-sensitive)
- Post-prunning

# Algunas desventajas ...

- Cuidado: overfitting !
- Inestables: pequeños cambios en los datos pueden hacer que el árbol resultante cambie notablemente
- Encontrar el árbol óptimo es un problema NP-completo
- Hay que tener cuidado si el dataset no está balanceado

# Tips y consejos prácticos ...

- A mayor cantidad de variables aumentan las probabilidades de sobre-entrenamiento: usar max_depth y analizar las componentes de nuestro conjunto de datos (técnicas de reducción de dimensionalidad).
- Analizar los árboles resultantes: dan mucha información de qué es lo que está pasando (FSS) !.
- Balancear los datos.
- Puedo usarlos para hacer análisis post-mortem en otros problemas.
- Uno de los algoritmos más utilizados: Classification And Regression Trees, o CART (permite trabajar con problemas de regresión obteniendo el promedio de las instancias que quedan en cada hoja)

![](files/images/Classifiers_comparison.png)